<a href="https://colab.research.google.com/github/nicolenair/capstone/blob/master/anno_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install stanfordnlp
!pip install import_ipynb


In [0]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import os
import re
import stanfordnlp
import import_ipynb

In [25]:
stanfordnlp.download('en')   # This downloads the English models for the neural pipeline
nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:13<00:00, 15.9MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/root/stanfordnlp_resources

In [0]:
# p = requests.get("https://en.wikipedia.org/wiki/Category:20th-century_British_women_writers")
# soup = BeautifulSoup(p.content, "html.parser")
# authors = soup.findAll("div", { "class" : "mw-category" })[0].text
# auth_list = [i for i in authors.split("\n") if len(i)>1]

# new_url = p + "&pagefrom=auth_list%2C+Priyanga%0APriyanga+Burford"

In [0]:
books = ["gables", "avonlea", "island", "dreams", "rilla" ,"ingleside", "poplars", "rainbow"]
urls = ["https://www.gutenberg.org/files/45/45-0.txt", "https://www.gutenberg.org/files/47/47-0.txt", "https://www.gutenberg.org/files/51/51-0.txt", "http://www.gutenberg.org/cache/epub/544/pg544.txt", "http://www.gutenberg.org/cache/epub/3796/pg3796.txt", "http://gutenberg.net.au/ebooks01/0100281.txt", "http://gutenberg.net.au/ebooks01/0100251.txt", "http://www.gutenberg.org/files/5343/5343-0.txt"]

for i in range(len(books)):
    page = requests.get(urls[i])
    file = open("{}.txt".format(books[i]) , "w+")
    file.write(page.text)
    file.close()

file = open("gables.txt", "r")
gables = file.read()
file = open("avonlea.txt", "r")
avonlea = file.read()
file = open("island.txt", "r")
island = file.read()
file = open("dreams.txt", "r")
dreams = file.read()
file = open("rilla.txt", "r")
rilla = file.read()
file = open("ingleside.txt", "r")
ingleside = file.read()
file = open("poplars.txt", "r")
poplars = file.read()
file = open("rainbow.txt", "r")
rainbow = file.read()

In [0]:
#agent

def return_nsubj(book, character="Anne"):
  sentences_with_agent_Anne = []
  sentences_with_agent = []
  if book=="gables":
    doc = nlp(gables)
  elif book=="dreams":
    doc = nlp(dreams)
  elif book=="island":
    doc = nlp(island)
  elif book=="avonlea":
    doc = nlp(avonlea)
  elif book=="rilla":
    doc = nlp(rilla)
  elif book=="ingleside":
    doc = nlp(ingleside)
  elif book=="poplars":
    doc = nlp(poplars)
  elif book=="rainbow":
    doc = nlp(rainbow)
  for i in list(doc.sentences):
    for e in i.dependencies:
  #       print(e[1])
      if e[1]=="nsubj" or e[1]=="agent" :
        if e[2].text==character:
          sentences_with_agent_Anne.append(" ".join([i.dependencies[a][2].text for a in range(len(i.dependencies)) if i.dependencies[a][2].text!=character]))
        else:
          sentences_with_agent.append(" ".join([i.dependencies[a][2].text for a in range(len(i.dependencies)) if i.dependencies[a][2].text!=character]))
        break
  target = []
  target.extend([[1] for i in range(len(sentences_with_agent_Anne))])
  target.extend([[0] for i in range(len(sentences_with_agent))])
  sentences = []
  sentences.extend([[i] for i in sentences_with_agent_Anne])
  sentences.extend([[i] for i in sentences_with_agent])
  return sentences, target

In [33]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# full_para_list_gables, target_gables = return_nsubj("gables")
# full_para_list_dreams, target_dreams = return_nsubj("dreams")
# full_para_list_avonlea, target_avonlea = return_nsubj("avonlea")
# full_para_list_island, target_island = return_nsubj("island")
# full_para_list_rilla, target_rilla = return_nsubj("rilla")
# full_para_list_ingleside, target_ingleside= return_nsubj("ingleside")
# full_para_list_poplars, target_poplars = return_nsubj("poplars")
# full_para_list_rainbow, target_rainbow = return_nsubj("rainbow")



In [35]:
os.listdir()

['Untitled0.ipynb',
 'Untitled1.ipynb',
 'Untitled2.ipynb',
 'Copy of CS146 Ass3.ipynb',
 'Untitled3.ipynb',
 'CS146 Ass3.ipynb',
 'cs146-7.1-pre-class-work.ipynb',
 'Copy of Untitled0.ipynb',
 'Copy of Rita CS152, 7.2.ipynb',
 'Copy of 7.2Preclass JW.ipynb',
 'Copy of CS146_LBA.ipynb',
 'Untitled4.ipynb',
 'Copy of Assignment 2 - Elisa Heinrich Mora',
 'Copy of #2 drafts.ipynb',
 'g.ipynb',
 'Copy of Final Project - CS152',
 'ExtraCS146.ipynb',
 'CS146.ipynb',
 'GPy.ipynb',
 'Copy of WumpusWorldFinals.ipynb',
 'CS166Assignment1.ipynb',
 'Modified copy of CS166Assignment1.ipynb',
 'Copy of 18 January copy of CS166Assignment1.ipynb',
 '18 January copy of CS166Assignment1.ipynb',
 'Copy 12:25 pm on Jan 20 CS166Assignment1.ipynb',
 'Untitled5.ipynb',
 'CS166 2.2.ipynb',
 'Untitled',
 'Untitled6.ipynb',
 'pyswip test notebook.ipynb',
 'forest fire non-interactive.ipynb',
 'Copy of forest fire non-interactive - with i (prob of catching on fire).ipynb',
 'cs166 pre-class 4.1.ipynb',
 'Copy o

In [0]:
# write to file
file = open("gables_sentence_list.txt", "w")
for i in range(len(full_para_list_gables)):
  file.write(full_para_list_gables[i][0])
  file.write("\n")
  file.write(str(target_gables[i][0]))
  file.write("\n")
file.close()
  
#write to file
file = open("dreams_sentence_list.txt", "w")
for i in range(len(full_para_list_dreams)):
  file.write(full_para_list_dreams[i][0])
  file.write("\n")
  file.write(str(target_dreams[i][0]))
  file.write("\n")
file.close()

#write to file
file = open("avonlea_sentence_list.txt", "w+")
for i in range(len(full_para_list_avonlea)):
  file.write(full_para_list_avonlea[i][0])
  file.write("\n")
  file.write(str(target_avonlea[i][0]))
  file.write("\n")
file.close()

file = open("island_sentence_list.txt", "w+")
for i in range(len(full_para_list_island)):
  file.write(full_para_list_island[i][0])
  file.write("\n")
  file.write(str(target_island[i][0]))
  file.write("\n")
file.close()

file = open("poplars_sentence_list.txt", "w+")
for i in range(len(full_para_list_poplars)):
  file.write(full_para_list_poplars[i][0])
  file.write("\n")
  file.write(str(target_poplars[i][0]))
  file.write("\n")
file.close()

file = open("rilla_sentence_list.txt", "w+")
for i in range(len(full_para_list_rilla)):
  file.write(full_para_list_rilla[i][0])
  file.write("\n")
  file.write(str(target_rilla[i][0]))
  file.write("\n")
file.close()

file = open("rainbow_sentence_list.txt", "w+")
for i in range(len(full_para_list_rainbow)):
  file.write(full_para_list_rainbow[i][0])
  file.write("\n")
  file.write(str(target_rainbow[i][0]))
  file.write("\n")
file.close()

file = open("ingleside_sentence_list.txt", "w+")
for i in range(len(full_para_list_ingleside)):
  file.write(full_para_list_ingleside[i][0])
  file.write("\n")
  file.write(str(target_ingleside[i][0]))
  file.write("\n")
file.close()
